In [44]:
import pandas as pd
import numpy as np

In [51]:



df_regAo = pd.read_excel("./../../data/RegionA_online.xlsx")
df_regAp = pd.read_excel("./../../data/RegionA_phone.xlsx")
df_regAs = pd.read_excel("./../../data/RegionA_store.xlsx")
df_regBo = pd.read_excel("./../../data/RegionB_online.xlsx")
df_regBp = pd.read_excel("./../../data/RegionB_phone.xlsx")
df_regBs = pd.read_excel("./../../data/RegionB_store.xlsx")
df_regCo = pd.read_excel("./../../data/RegionC_online.xlsx")
df_regCp = pd.read_excel("./../../data/RegionC_phone.xlsx")
df_regCs = pd.read_excel("./../../data/RegionC_store.xlsx")
df_regDo = pd.read_excel("./../../data/RegionD_online.xlsx")
df_regDp = pd.read_excel("./../../data/RegionD_phone.xlsx")
df_regDs = pd.read_excel("./../../data/RegionD_store.xlsx")

df_regAo.duplicated().sum()

0

In [63]:
df_regDs.duplicated().sum()

0

In [41]:
df_preis = (
    pd.read_excel("./../../data/Preise.xlsx")
        .rename(columns={"Jahr": "year",
                         "preis": "price"})
        .assign(year=lambda _data: pd.to_numeric(_data["year"].str.strip('"')))
        .replace([20022, "Tablet Basics"],[2022, "Tablet Basic"])
        
)

In [34]:
df_dt = (
    pd.read_excel("./../../data/Downtime.xlsx")
        .rename(columns={"Unnamed: 0": "year",
                         "Region A": "a",
                         "Region B": "b",
                         "Region C": "c",
                         "Region D": "d"})
)
df_dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    2 non-null      int64  
 1   a       2 non-null      float64
 2   b       2 non-null      float64
 3   c       2 non-null      float64
 4   d       2 non-null      float64
dtypes: float64(4), int64(1)
memory usage: 208.0 bytes


In [104]:
df_crm = (
    pd.read_excel("./../../data/CRM_data.xlsx")
        .rename(columns={"id": "transaction_id",
                        "vorname": "firstname",
                         "nachname": "lastname"})
)

In [105]:
df_crm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340000 entries, 0 to 339999
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   transaction_id   340000 non-null  int64  
 1   firstname        340000 non-null  object 
 2   lastname         340000 non-null  object 
 3   iban             340000 non-null  object 
 4   email            340000 non-null  object 
 5   customer_rating  340000 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 15.6+ MB


In [21]:
df_dt

,year,a,b,c,d
0,2021,0.10,0.01,0.05,0.00
1,2022,0.15,0.15,0.04,0.01


In [42]:
df_preis

,year,product,price
0,2021,Tablet Pro,999
1,2022,Tablet Pro,999
2,2021,Tablet Mini,680
3,2022,Tablet Mini,550
4,2021,Tablet Basic,529
5,2022,Tablet Basic,499


In [117]:
df_regAo = (
    pd.read_excel("./../../data/RegionA_online.xlsx")
        .rename(columns={"Tablet Basic": "basic",
                         "Tablet Mini": "mini",
                         "Tablet Pro": "pro",
                         "kosten_mini": "cost_mini",
                         "kosten_basic": "cost_basic",
                         "kosten_pro": "cost_pro"})
        .assign(year=lambda _data: _data["date"].dt.year,
                month=lambda _data: _data["date"].dt.month,
                week=lambda _data: _data["date"].dt.isocalendar().week,
                cost_basic=lambda _data: np.round(_data["cost_basic"], 2),
                cost_mini=lambda _data: np.round(_data["cost_mini"], 2),
                cost_pro=lambda _data: np.round(_data["cost_pro"], 2),
                cost_per_unit=lambda _data: _data["cost_basic"].fillna(0) + _data["cost_mini"].fillna(0) + _data["cost_pro"].fillna(0),
                cost=lambda _data: _data["cost_per_unit"] * _data["amount"]
                )
        .merge(df_preis, on=["year", "product"])
        .assign(revenue=lambda _data: _data["price"] * _data["amount"],
                profit_per_unit=lambda _data: _data["price"] - _data["cost_per_unit"],
                profit=lambda _data: _data["profit_per_unit"] * _data["amount"])
        .merge(df_crm[["customer_rating", "transaction_id"]], on="transaction_id")
)

In [118]:
df_regAo

,date,product,discount,amount,basic,mini,pro,cost_basic,cost_mini,cost_pro,...,year,month,week,cost_per_unit,cost,price,revenue,profit_per_unit,profit,customer_rating
0,2021-11-01,Tablet Basic,1.0,5,1,0,0,170.02,NaN,NaN,...,2021,11,44,170.02,850.10,529,2645,358.98,1794.90,7.1
1,2021-11-01,Tablet Basic,1.0,7,1,0,0,170.02,NaN,NaN,...,2021,11,44,170.02,1190.14,529,3703,358.98,2512.86,5.9
2,2021-11-01,Tablet Basic,0.9,7,1,0,0,210.03,NaN,NaN,...,2021,11,44,210.03,1470.21,529,3703,318.97,2232.79,7.5
3,2021-11-01,Tablet Basic,1.0,10,1,0,0,170.08,NaN,NaN,...,2021,11,44,170.08,1700.80,529,5290,358.92,3589.20,6.6
4,2021-11-01,Tablet Basic,1.0,4,1,0,0,170.09,NaN,NaN,...,2021,11,44,170.09,680.36,529,2116,358.91,1435.64,7.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8525,2022-10-31,Tablet Mini,1.0,4,0,1,0,NaN,136.09,NaN,...,2022,10,44,136.09,544.36,550,2200,413.91,1655.64,8.3
8526,2022-10-31,Tablet Mini,1.0,8,0,1,0,NaN,136.07,NaN,...,2022,10,44,136.07,1088.56,550,4400,413.93,3311.44,8.2
8527,2022-10-31,Tablet Mini,1.0,3,0,1,0,NaN,136.07,NaN,...,2022,10,44,136.07,408.21,550,1650,413.93,1241.79,6.4
8528,2022-10-31,Tablet Mini,1.0,2,0,1,0,NaN,136.06,NaN,...,2022,10,44,136.06,272.12,550,1100,413.94,827.88,7.5


In [84]:
df_regAo = df_regAo.merge(df_preis, on=["year", "product"])

In [85]:
index = df_regAo["pro"].nlargest(5).index
df_regAo.loc[index]

,date,product,discount,amount,basic,mini,pro,cost_basic,cost_mini,cost_pro,transaction_id,year,price
1719,2021-11-02,Tablet Pro,1.0,4,0,0,1,NaN,NaN,645.80,1301,2021,999
1720,2021-11-02,Tablet Pro,1.0,8,0,0,1,NaN,NaN,685.84,1353,2021,999
1721,2021-11-02,Tablet Pro,1.0,5,0,0,1,NaN,NaN,685.85,1375,2021,999
1722,2021-11-03,Tablet Pro,1.0,8,0,0,1,NaN,NaN,686.11,1804,2021,999
1723,2021-11-03,Tablet Pro,1.0,7,0,0,1,NaN,NaN,686.12,1817,2021,999
